## Problem Set 4
### UGBA 88: Data and Decisions, Fall 2019

In [ ]:
#run this cell once, then *restart kernel*
%pip install gsExport

In [ ]:
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import numpy as np
import gsExport

Deadline: This assignment is due Monday, October 14th at noon (12pm). Late work will not be accepted.

You will submit your solutions using both OKpy and Gradescope. You will find detailed submission instructions ([here](https://docs.google.com/document/d/1vrg66vGtBf93xt4-LUQPpacUAQAxIJEeJ10fRsb8oUc/)). **Please do not remove or add cells and please ignore the '#newpage' cells** (these are here to facilitate Gradescope submission).

You should start early so that you have time to get help if you're stuck. Post questions on [Piazza](https://piazza.com/class/jzw0f05ebpof0). Check the syllabus  for the office hours schedule. Remember that Connector Assistant office hours are for *coding questions only*.

#newpage

## Question 1: Job Search Assistance

**(25 points)** In many industrialized countries, professional job counseling services are a popular tool for helping educated young people find stable jobs. By connecting employers with appropriately matched job seekers, counseling agencies are thought to improve the process of finding work and generate better matches between employers and employees.

Crepon et al. (2013) study a large-scale, privately-run job seeker assistance program targeted at young, educated job seekers in France. Under the program, private agencies are contracted to provide intensive placement services to young graduates (with at least a two-year college degree) who have been unemployed for at least six months. To incentivize agency performance, private providers are paid a bonus if a job seeker they assist finds a job with a contract of at least six months and stays employed for at least six months.

The researchers study the assistance program using a randomized experiment. The experiment covers about 30,000 job seekers in 235 cities. Job seekers typically look for jobs in the city they live in. Job seekers assigned to the treatment are offered the opportunity to enroll in the privately-run job placement program. Control job seekers cannot enroll in the job placement program, but still have access to the French public employment agency, which is thought to provide less intensive placement services. Existing evidence suggests that the private job placement program will improve the labor market outcomes of participants.

The main outcome of interest is whether a job seeker has found a stable job after some period folowing their treatment assignment.

**a. (5 points)** Suppose the no interference assumption is satisfied. Given the outcome and treatment described in the text, describe the following term in plain English:

$$E[Y_{0i} | D_{i} = 1]$$

*Write answer here*

**b. (5 points)** How might interference undermine our analysis? Would you expect spillovers to *increase* or *decrease* treatment effect estimates, and why? [Note: there is more than one "correct" answer to this question.]

*Write answer here*

**c. (5 points)** What randomization strategy could you use to account for spillovers? Why would this help, and how would you implement this strategy?

*Write answer here*

**d. (5 points)** How would you design the experiment to *measure* spillovers? What comparison in the data would allow you to measure spillovers?

*Write answer here*

**e. (5 points)** The researchers measure the outcome of whether a job seeker has found a stable job using a survey. One concern is that the survey response rate is 70%, significantly below 100%. In particular, the researchers are concerned that job seekers who have *not* found a stable job will be less likely to respond to the survey.

Why might this be an issue for the experiment analysis? Would you expect this issue to add a *positive* or *negative* bias to the treatment effect estimate? Explain your reasoning.

*Write answer here*

#newpage

## Question 2: Bounding Treatment Effects Under Attrition

**(25 points)** In Lab 2, we discovered an **attrition** problem in the Ctrip experiment. We wanted to measure the causal effect of Work From Home on worker productivity, and found that control group employees were more likely than treatment group employees to leave the company by the end of the experiment. This presents a problem because, while randomization guarantees that the treatment and control groups are comparable at the *beginning of the experiment*, attrition breaks this guarantee so that by the *end of the experiment* the treatment and control employees that remain may no longer be comparable.

In this problem, we will apply the **trimming** method to estimating treatment effects in the presence of attrition. Under a few additional assumptions, trimming allows us to put bounds on the average treatment effect for a subset of the population. That is, we will be able to say this average treatment effect is no larger (more positive) than some **upper bound**, and no smaller (more negative) than some **lower bound**.

The assumption we will make is that, for any given employee, the treatment can only *increase* the chances that an employee stays at the company. This assumption is untestable; while we know from Lab 2 that the treatment increases the chances than an employee stays at the company *on average*, we do not know whether this is true *for every employee individually*.

Under this assumption, there are **three** types of employees that begin in the experiment: those that will stay at the company regardless of whether they are assigned to the treatment or control group (**'Always-Stayers'**); those that will leave the company regardless (**‘Always-Leavers’**); and those that will only stay at the company if they are assigned to the treatment group (**‘If-Treated-Stayers’**). 

By the way we’ve defined these groups, we know that the control group contains only Always-Stayers by the end of the experiment. That's because the Always-Leavers and If-Treated-Stayers have left the company. By contrast, the treatment group contains both Always-Stayers and If-Treated-Stayers by the end of the experiment. Attrition introduces bias because treatment versus control comparisons are no longer comparing like with like; in particular, the treatment group includes If-Treated-Stayers and the control group does not.

The idea of trimming is that, if we can remove the If-Treated-Stayers from our treatment group, then only Always-Stayers remain in both the treatment and control groups. That would *solve* the attrition problem. Once we've removed the If-Treated-Stayers, the comparison of the treatment and control groups provides an estimate of the average treatment effect among Always-Stayers. *The problem is, we do not know which employee is an If-Treated-Stayer.* This is the fundamental problem of causal inference all over again; we know whether a treatment or control employee leaves the company or not, but we do not know what *would have happened* if they were assigned to the other group.

However, we can calculate bounds on the average treatment effect for Always-Stayers by considering **best case** and **worst case**  scenarios for which treatment group employees are the If-Treated-Stayers. In the **best case** scenario, we assume the *least productive* treatment group employees that stay at the company are the If-Treated-Stayers. This is the best case because removing the lowest performers from the treatment group will *increase* average productivity in the (remaining) treatment group and hence make the treatment group look even better compared to the control group. Removing these employees from the treatment group and comparing the modified treatment group to the control group provides an **upper bound** on the average treatment effect for  Always-Stayers. For the **worst case** scenario, we assume the *most productive* treatment group employees that stay at the company are the If-Treated-Stayers, and remove them from the data. Comparing the modified treatment group to the control group provides a **lower bound** on the average treatment effect for  Always-Stayers.

Run the cell below to read in the experiment data.

In [ ]:
experiment = Table.read_table('wfh_exp_att.csv')
experiment.show(5)

While the experiment data from the last problem set summarizes productivity over the full experiment, these data only cover the last 20 days of the experiment, after many employees have left the company. Employees that leave the company before this period are excluded from the data. Recall that `performance` is employee productivity measured over this period (as in PS3 and Lab 2). (`performance` summarizes a variety of performance metrics, including the number of phone calls an employee answers, the number of orders taken, and the number of reservation phone calls made. The measure is scaled to have mean 0 and standard deviation 1 across employees. That is, a one unit increase in `performance` coincides with a one standard deviation increase in productivity. For reference, a one standard deviation increase in phone calls made per week is an increase of about 100 phone calls.)

**a. (5 points)** Calculate the difference in average `performance` between the treatment and control groups. Print your results.

In [ ]:
#write code here

In the next question, you will estimate the number of employees in the treatment group that are If-Treated-Stayers. You will estimate this number by measuring the "excess" attrition we see in the control group relative to the treatment group. Let $n_{1}$ denote the number of treatment group employees at the beginning of the experiment, and $n_{1}^{\text{end}}$ denote the number of treatment group employees that remain at the company by the end of the experiment. Let $n_{0}$ and $n_{0}^{\text{end}}$ denote analogous numbers for the control group employees. The estimated  share of treatment group that are If-Treated-Stayers is given by

$$ \pi = \frac{\frac{n_{1}^{\text{end}}}{n_{1}} - \frac{n_{0}^{\text{end}}}{n_{0}}}{n_{1}^{\text{end}}/n_{1}}$$

To get the estimated number of employees that are in the treatment group, multiply this share $\pi$ by $n_{1}^{\text{end}}$, the number of treatment group employees that remain at the company by the end of the experiment, and round the result to the nearest whole number.

In this experiment, $n_{0} = 114$ and $n_{1} = 126$.

**b. (5 points)** Estimate the number of employees in the treatment group that are If-Treated-Stayers. Print your output and write your rounded estimate.

In [ ]:
#Write code here

*Write rounded answer here*

**c. (5 points)** Calculate the **upper bound** for the average treatment effect for Always-Stayers. First, remove the bottom $k$ least productive employees from the treatment group, where $k$ is your answer to **part (b)**. Then, calculate the difference in average performance between the modified treatment group and the control group. Make sure to print your results.

In [ ]:
#write answer here

**d. (5 points)** Calculate the **lower bound** for the average treatment effect for Always-Stayers. Make sure to print your results.

In [ ]:
#write answer here

**e. (5 points)** Given our findings in Lab 2, the researchers argue that the true average treatment effect for Always-Stayers is likely closer to the upper bound than the lower bound. Why?

*write answer here* 

#newpage

## Submission

Before submitting, please click "Kernel" above and click "Restart & Run All" to ensure all of your code is working as expected. This is important. Code that does not run cannot be graded. After confirming that all of your work looks and runs as you'd like it to, run **BOTH** of the below cells to submit your work.

Make sure that the following runs successfully for submission to OkPy.

In [ ]:
from client.api.notebook import Notebook
ok = Notebook('pset4.ok')                
_ = ok.auth(inline=True)
_ = ok.submit()

Then, make sure that the following runs successfully to generate a PDF to upload to Gradescope. **Do not upload any other PDF to Gradescope other than the one generated by the below code.** If you have difficulty downloading the PDF, please review the submission instructions ([here](https://docs.google.com/document/d/1vrg66vGtBf93xt4-LUQPpacUAQAxIJEeJ10fRsb8oUc/)) or see Piazza for troubleshooting steps.


In [ ]:
gsExport.generateSubmission('pset4.ipynb')